In [2]:
!gdown 1oHoYT7J4-xKfNu6cfBOMKHyO0QBqdYsI
!unzip /content/calibration_data2.zip -d /content/calibration_data

Downloading...
From: https://drive.google.com/uc?id=1oHoYT7J4-xKfNu6cfBOMKHyO0QBqdYsI
To: /content/calibration_data2.zip
100% 134M/134M [00:02<00:00, 51.9MB/s]
Archive:  /content/calibration_data2.zip
replace /content/calibration_data/data/ILSVRC2012_val_00000001.JPEG? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [3]:
!ls /content/


calibration_data  calibration_data2.zip  sample_data


In [1]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.quantization.qconfig import QConfig
from torch.quantization.observer import MinMaxObserver, MovingAverageMinMaxObserver, MovingAveragePerChannelMinMaxObserver, PerChannelMinMaxObserver
import yaml
from torch.ao.quantization.fake_quantize import FakeQuantize
import torch
import torchvision
import os
import torch.nn as nn
import torch.ao.quantization.quantize_fx as quantize_fx
import copy

In [2]:
  example_inputs = (torch.randn(1, 3, 224, 224),)
  model_fp = torchvision.models.resnet18(pretrained=True)
  model_to_quantize = copy.deepcopy(model_fp)
  quantize_fx.fuse_fx(model_to_quantize.eval())
  transform_cali = transforms.Compose([
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.ToTensor(),  # Convert the image to a PyTorch tensor
      transforms.Normalize(
          mean=[0.485, 0.456, 0.406],  # ImageNet dataset mean
          std=[0.229, 0.224, 0.225]  # ImageNet dataset standard deviation
      )
  ])
  transform_val = transforms.Compose([
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.ToTensor(),  # Convert the image to a PyTorch tensor
      transforms.Normalize(
          mean=[0.485, 0.456, 0.406],  # ImageNet dataset mean
          std=[0.229, 0.224, 0.225]  # ImageNet dataset standard deviation
      )
  ])


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
from PIL import Image
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import random
import os

class MyDataset(Dataset):
    def __init__(self, txt_file, transform=None, dir=None):
        self.data = []
        with open(txt_file, 'r') as f:
            for line in f:
                image_path = line.split(' ')[0]
                label = line.split(' ')[1].split('\n')[0]
                self.data.append((image_path, int(label)))
        self.transform = transform
        self.dir = dir
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path, label = self.data[idx]
        if self.dir:
            image_path = os.path.join(self.dir, image_path)
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label


class CustomDataset(Dataset):
    def __init__(self, original_dataset, samples_per_epoch):
        self.original_dataset = original_dataset
        self.samples_per_epoch = samples_per_epoch

    def __getitem__(self, index):
        random_index = random.randint(0, len(self.original_dataset) - 1)
        return self.original_dataset[random_index]

    def __len__(self):
        return self.samples_per_epoch

In [4]:
val_dataset = MyDataset('/content/calibration_data/samples.txt',
                        transform=transform_val)

In [5]:
print(len(val_dataset))

896


In [6]:
qconfig = QConfig(activation=FakeQuantize.with_args(observer=MinMaxObserver,
                                                          quant_min=0,
                                                          quant_max=255,
                                                          qscheme=torch.per_tensor_affine,
                                                          reduce_range=False,),
                        weight=FakeQuantize.with_args(observer=PerChannelMinMaxObserver,
                                                      quant_min=-128,
                                                      quant_max=127,
                                                      dtype=torch.qint8,
                                                      qscheme=torch.per_channel_symmetric,
                                                      #per_tensor_symmetric
                                                      reduce_range=False,
                                                      ch_axis=0,
                                                        ))

In [7]:
from torch.ao.quantization import (
  get_default_qconfig_mapping,
  get_default_qat_qconfig_mapping,
  QConfigMapping,
)
qconfig_mapping = QConfigMapping().set_global(qconfig)

In [8]:
model_prepared = quantize_fx.prepare_fx(model_to_quantize, qconfig_mapping, example_inputs)

In [9]:
criterion = nn.CrossEntropyLoss()

In [10]:
def evaluate_model(model, validation_dataloader, criterion, device):
    model.to(device)
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for data in validation_dataloader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)

            # Compute the loss
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = running_loss / len(validation_dataloader)
    accuracy = 100 * correct / total

    return avg_loss, accuracy

In [11]:
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=False, num_workers=8, drop_last=True)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [12]:
loss, acc = evaluate_model(model_prepared, val_loader, criterion,'cuda')

In [13]:
acc

69.75

In [14]:
model_prepared = model_prepared.to('cpu')

In [15]:
model_quantized = quantize_fx.convert_fx(model_prepared)

In [16]:
loss, acc = evaluate_model(model_quantized, val_loader, criterion,'cpu')

In [17]:
acc

66.625

In [18]:
model_prepared

GraphModule(
  (activation_post_process_0): FakeQuantize(
    fake_quant_enabled=tensor([1], dtype=torch.uint8), observer_enabled=tensor([1], dtype=torch.uint8), quant_min=0, quant_max=255, dtype=torch.quint8, qscheme=torch.per_tensor_affine, ch_axis=-1, scale=tensor([0.0187]), zero_point=tensor([114], dtype=torch.int32)
    (activation_post_process): MinMaxObserver(min_val=-2.1179039478302, max_val=2.640000104904175)
  )
  (conv1): ConvReLU2d(
    (0): QuantizedConv2d(Reference)(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): ReLU(inplace=True)
  )
  (activation_post_process_1): FakeQuantize(
    fake_quant_enabled=tensor([1], dtype=torch.uint8), observer_enabled=tensor([1], dtype=torch.uint8), quant_min=0, quant_max=255, dtype=torch.quint8, qscheme=torch.per_tensor_affine, ch_axis=-1, scale=tensor([0.0273]), zero_point=tensor([0], dtype=torch.int32)
    (activation_post_process): MinMaxObserver(min_val=0.0, max_val=6.968907356262207)
  )
  (maxpool): MaxPool2d(kern

#TASK:
1. Split the 1000 samples into two groups, 50%,50%, one of them will be named calibration_dataset while the other will be validation_dataset. You should use calibration dataset to calibrate scale and zero point before performing actual quantization.
2. Make PTQ training for MobileNetV2, using PerchannelMinMax Quantization for weights and PerTensorMinMax Quantization separately.
3. Make Perchannel MovingAverageMinMax Quantization for weights and MovingAverage Pertensor Quantization for activation. Compare to results in 2., Check which one is better. Compare some scales and zero points of from the observers. Explain why one of the solution is better?

In [49]:
import copy
import random
# val_dataset = MyDataset('samples.txt', transform=transform_val)
calibration_dataset = copy.deepcopy(val_dataset)
random.seed(2137)
random.shuffle(calibration_dataset.data)
validation_dataset = copy.deepcopy(calibration_dataset)
validation_dataset.data = validation_dataset.data[:len(validation_dataset)//2]
calibration_dataset.data = calibration_dataset.data[len(calibration_dataset)//2:]
calibration_loader = DataLoader(calibration_dataset, batch_size=100, shuffle=False, num_workers=8, drop_last=True)
validation_loader = DataLoader(validation_dataset, batch_size=100, shuffle=False, num_workers=8, drop_last=True)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [50]:
print(len(validation_dataset), len(calibration_dataset))

448 448


In [51]:
model_to_quantize = copy.deepcopy(model_fp)
quantize_fx.fuse_fx(model_to_quantize.eval())
model_prepared = quantize_fx.prepare_fx(model_to_quantize, qconfig_mapping, example_inputs)
loss, acc = evaluate_model(model_prepared, calibration_loader, criterion,'cuda')

In [52]:
model_prepared = model_prepared.to('cpu')
model_quantized = quantize_fx.convert_fx(model_prepared)
loss, acc = evaluate_model(model_prepared, validation_loader, criterion,'cuda')

In [53]:
print("calibrated:", loss, acc)
# calibration seems to improve the accuracy
# lets compare it with no quantization

calibrated: 1.3138143718242645 66.0


In [54]:
model_to_quantize = copy.deepcopy(model_fp)
quantize_fx.fuse_fx(model_to_quantize.eval())
loss, acc = evaluate_model(model_to_quantize, validation_loader, criterion,'cuda')

In [55]:
print("no quantization (fused)", loss, acc)


no quantization (fused) 1.1694874167442322 69.75


# MobileNetV2

In [ ]:
  
model_fp = torchvision.models.quantization.mobilenet_v2(pretrained=True)
model_to_quantize = copy.deepcopy(model_fp)
quantize_fx.fuse_fx(model_to_quantize.eval())


In [57]:
qconfig = QConfig(activation=FakeQuantize.with_args(observer=MinMaxObserver,
                                                          quant_min=0,
                                                          quant_max=255,
                                                          qscheme=torch.per_tensor_affine,
                                                          reduce_range=False,),
                        weight=FakeQuantize.with_args(observer=PerChannelMinMaxObserver,
                                                      quant_min=-128,
                                                      quant_max=127,
                                                      dtype=torch.qint8,
                                                      qscheme=torch.per_channel_symmetric,
                                                      #per_tensor_symmetric
                                                      reduce_range=False,
                                                      ch_axis=0,
                                                        ))
qconfig_mapping = QConfigMapping().set_global(qconfig)
model_prepared = quantize_fx.prepare_fx(model_to_quantize, qconfig_mapping, example_inputs)

In [58]:
loss, acc = evaluate_model(model_prepared, calibration_loader, criterion,'cuda')

In [59]:
model_prepared = model_prepared.to('cpu')
model_quantized_per_channel_minmax = quantize_fx.convert_fx(model_prepared)

In [60]:
loss, acc = evaluate_model(model_quantized_per_channel_minmax, validation_loader, criterion,'cpu')

In [61]:
print("per channel observer: ", loss, acc)

per channel observer:  1.7089119255542755 58.5


In [62]:
qconfig = QConfig(activation=FakeQuantize.with_args(observer=MinMaxObserver,
                                                          quant_min=0,
                                                          quant_max=255,
                                                          qscheme=torch.per_tensor_affine,
                                                          reduce_range=False,),
                        weight=FakeQuantize.with_args(observer=MinMaxObserver,
                                                      quant_min=-128,
                                                      quant_max=127,
                                                      dtype=torch.qint8,
                                                      qscheme=torch.per_tensor_symmetric,
                                                      reduce_range=False,
                                                        ))
qconfig_mapping = QConfigMapping().set_global(qconfig)
model_to_quantize = copy.deepcopy(model_fp)
quantize_fx.fuse_fx(model_to_quantize.eval())
model_prepared = quantize_fx.prepare_fx(model_to_quantize, qconfig_mapping, example_inputs)

In [63]:
loss, acc = evaluate_model(model_prepared, calibration_loader, criterion,'cuda')
model_prepared = model_prepared.to('cpu')
model_quantized_minmax_per_tensor = quantize_fx.convert_fx(model_prepared)
loss, acc = evaluate_model(model_quantized_minmax_per_tensor, validation_loader, criterion,'cpu')
print("per tensor observer: ", loss, acc)


per tensor observer:  1.3971586227416992 66.0


In [64]:
qconfig = QConfig(activation=FakeQuantize.with_args(observer=MovingAverageMinMaxObserver,
                                                          quant_min=0,
                                                          quant_max=255,
                                                          qscheme=torch.per_tensor_affine,
                                                          reduce_range=False,),
                        weight=FakeQuantize.with_args(observer=MovingAveragePerChannelMinMaxObserver,
                                                      quant_min=-128,
                                                      quant_max=127,
                                                      dtype=torch.qint8,
                                                      qscheme=torch.per_channel_symmetric,
                                                      reduce_range=False,
                                                      ch_axis=0,
                                                        ))
qconfig_mapping = QConfigMapping().set_global(qconfig)
model_to_quantize = copy.deepcopy(model_fp)
quantize_fx.fuse_fx(model_to_quantize.eval())
model_prepared = quantize_fx.prepare_fx(model_to_quantize, qconfig_mapping, example_inputs)

In [65]:
loss, acc = evaluate_model(model_prepared, calibration_loader, criterion,'cuda')
model_prepared = model_prepared.to('cpu')
model_quantized_moving_average_per_channel = quantize_fx.convert_fx(model_prepared)
loss, acc = evaluate_model(model_quantized_moving_average_per_channel, validation_loader, criterion,'cpu')
print("moving average per channel observer: ", loss, acc)

moving average per channel observer:  1.6461626291275024 60.5


In [66]:
qconfig = QConfig(activation=FakeQuantize.with_args(observer=MovingAverageMinMaxObserver,
                                                          quant_min=0,
                                                          quant_max=255,
                                                          qscheme=torch.per_tensor_affine,
                                                          reduce_range=False,),
                        weight=FakeQuantize.with_args(observer=MovingAverageMinMaxObserver,
                                                      quant_min=-128,
                                                      quant_max=127,
                                                      dtype=torch.qint8,
                                                      qscheme=torch.per_tensor_affine,
                                                      reduce_range=False,
                                                        ))
qconfig_mapping = QConfigMapping().set_global(qconfig)
model_to_quantize = copy.deepcopy(model_fp)
quantize_fx.fuse_fx(model_to_quantize.eval())
model_prepared = quantize_fx.prepare_fx(model_to_quantize, qconfig_mapping, example_inputs)

In [67]:
loss, acc = evaluate_model(model_prepared, calibration_loader, criterion,'cuda')
model_prepared = model_prepared.to('cpu')
model_quantized_moving_average_per_tensor = quantize_fx.convert_fx(model_prepared)
loss, acc = evaluate_model(model_quantized_moving_average_per_tensor, validation_loader, criterion,'cpu')
print("moving average per tensor (affine scheme) observer: ", loss, acc)

moving average per tensor (affine scheme) observer:  1.4038624167442322 66.5


Moving average observers yield better results in this case (I tried with a different seed and the conclusions were likewise). Per tensor moving average minmax,for this seed, is better than standard per tensor minmax by 0.5pp and per channel moving average is better than standard per channel by 2pp. 
I guess that's because moving average calibration tends to "ignore" the extreme Xmin and Xmax values, which results in a smaller [Xmin; Xmax] interval which results in a smaller scale parameter. Smaller scale parameter means denser quantization so the results tend to be more accurate.